<a href="https://colab.research.google.com/github/lsteffenel/NumbaCuda/blob/main/Noyaux_CUDA_en_Python_avec_Numba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Avant de commencer
L'exécution de ces notebooks sur Colab nécessite deux choses (au 4/2/2025) :

1. des resources GPU
  * Menu "Exécution" -> "Modifier le type d'exécution"
2. D'utiliser une version plus ancienne de Colab en raison de certaines incompatibilités du pilote Nvidia
  * Menu "Outils" -> "Pallette de commandes". Cherchez "version" dans la barre et sélectionnez l'option "Utiliser la version d'environnement d'exécution de remplacement"

# Kernels CUDA personnalisés en Python avec Numba

Dans cette section, nous approfondirons notre compréhension de la manière dont le modèle de programmation CUDA organise le travail parallèle et nous exploiterons cette compréhension pour écrire des **kernels** CUDA personnalisés, des fonctions qui s'exécutent en parallèle sur les GPU CUDA. Les kernels CUDA personnalisés, en utilisant le modèle de programmation CUDA, nécessitent plus de travail à mettre en œuvre que, par exemple, la simple décoration d'un ufunc avec `@vectorize`. Cependant, ils rendent possible le calcul parallèle dans des endroits où les ufuncs ne le peuvent tout simplement pas, et offrent une flexibilité qui peut conduire au plus haut niveau de performance.

## Pourquoi créer des Kernels Personnalisés ?

Les fonctions ufuncs sont incroyablement élégantes et, pour toute opération scalaire devant être effectuée élément par élément sur des données, elles constituent probablement l'outil idéal.

Comme vous le savez, il existe de nombreuses classes de problèmes, voire plus, qui ne peuvent être résolues en appliquant la même fonction à chaque élément d'un ensemble de données. Considérez, par exemple, tout problème qui nécessite l'accès à plusieurs éléments d'une structure de données afin de calculer sa sortie, comme les algorithmes de pochoir, ou tout problème qui ne peut pas être exprimé par un mappage d'une valeur d'entrée vers une valeur de sortie, comme une réduction. Beaucoup de ces problèmes sont toujours intrinsèquement parallélisables, mais ne peuvent pas être exprimés par une fonction ufunc.

L'écriture de kernels CUDA personnalisés, bien que plus difficile que l'écriture d'UFuncs accélérés par GPU, offre aux développeurs une flexibilité considérable pour les types de fonctions qu'ils peuvent envoyer pour s'exécuter en parallèle sur le GPU. De plus, comme vous commencerez à l'apprendre dans cette section et la suivante, il fournit également un contrôle précis sur la manière dont le parallélisme est effectué en exposant explicitement la hiérarchie des threads de CUDA aux développeurs.

Bien que nous restions purement en Python, la façon dont nous écrivons les noyaux CUDA à l'aide de Numba rappelle beaucoup la façon dont les développeurs les écrivent dans CUDA C/C++. Pour ceux d'entre vous qui connaissent la programmation en CUDA C/C++, vous apprendrez probablement très rapidement les noyaux personnalisés en Python avec Numba, et pour ceux d'entre vous qui les apprennent pour la première fois, sachez que le travail que vous faites ici vous sera également utile si jamais vous avez besoin ou souhaitez développer CUDA en C/C++, ou même, faire une étude de la richesse des ressources CUDA sur le Web qui représentent le plus souvent le code CUDA C/C++.

## Introduction aux kernels CUDA

Lors de la programmation en CUDA, les développeurs écrivent des fonctions pour le GPU appelées **kernels** ou **noyaux**, qui sont exécutées, ou dans le jargon CUDA, **lancées**, sur les nombreux cœurs du GPU dans des **threads** parallèles. Lorsque les noyaux sont lancés, les programmeurs utilisent une syntaxe spéciale, appelée **configuration d'exécution** (également appelée configuration de lancement) pour décrire la configuration de l'exécution parallèle.

Les diapositives suivantes (qui apparaîtront après l'exécution de la cellule ci-dessous) donnent une introduction de haut niveau sur la façon dont les noyaux CUDA peuvent être créés pour fonctionner sur de grands ensembles de données en parallèle sur le périphérique GPU. Parcourez les diapositives, puis vous commencerez à écrire et à exécuter vos propres noyaux CUDA personnalisés, en utilisant les idées présentées dans les diapositives.

In [ ]:
from IPython.display import IFrame
IFrame('https://view.officeapps.live.com/op/view.aspx?src=https://developer.download.nvidia.com/training/courses/C-AC-02-V1/AC_CUDA_Python_1.pptx', 640, 390)

## Un premier noyau CUDA

Commençons par un exemple concret et très simple en réécrivant notre fonction d'addition pour les tableaux NumPy 1D. Les noyaux CUDA sont compilés à l'aide du décorateur `numba.cuda.jit`. `numba.cuda.jit` ne doit pas être confondu avec le décorateur `numba.jit` que vous avez déjà appris et qui optimise les fonctions **pour le CPU**.

Nous commencerons par un exemple très simple pour mettre en évidence une partie de la syntaxe essentielle. Il convient de mentionner que cette fonction particulière pourrait en fait être écrite comme un ufunc, mais nous la choisissons ici pour garder l'accent sur l'apprentissage de la syntaxe. Nous passerons ci-dessous à des fonctions plus adaptées à l'écriture en tant que noyau personnalisé. Assurez-vous de lire attentivement les commentaires, car ils fournissent des informations importantes sur le code.

In [ ]:
from numba import cuda

# Note the use of an `out` array. CUDA kernels written with `@cuda.jit` do not return values,
# just like their C counterparts. Also, no explicit type signature is required with @cuda.jit
@cuda.jit
def add_kernel(x, y, out):

    # The actual values of the following CUDA-provided variables for thread and block indices,
    # like function parameters, are not known until the kernel is launched.

    # This calculation gives a unique thread index within the entire grid (see the slides above for more)
    idx = cuda.grid(1)          # 1 = one dimensional thread grid, returns a single value.
                                # This Numba-provided convenience function is equivalent to
                                # `cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x`

    # This thread will do the work on the data element with the same index as its own
    # unique index within the grid.
    out[idx] = x[idx] + y[idx]

In [ ]:
import numpy as np

n = 4096
x = np.arange(n).astype(np.int32) # [0...4095] on the host
y = np.ones_like(x)               # [1...1] on the host

d_x = cuda.to_device(x) # Copy of x on the device
d_y = cuda.to_device(y) # Copy of y on the device
d_out = cuda.device_array_like(d_x) # Like np.array_like, but for device arrays

# Because of how we wrote the kernel above, we need to have a 1 thread to one data element mapping,
# therefore we define the number of threads in the grid (128*32) to equal n (4096).
threads_per_block = 128
blocks_per_grid = 32

In [ ]:
add_kernel[blocks_per_grid, threads_per_block](d_x, d_y, d_out)
cuda.synchronize()
print(d_out.copy_to_host()) # Should be [1...4096]

### Exercice : modifier le code

Apportez les modifications mineures suivantes au code ci-dessus pour voir comment cela affecte son exécution. Faites des suppositions éclairées sur ce qui se passera avant d'exécuter le code :

* Diminuez la variable `threads_per_block`
* Diminuez la variable `blocks_per_grid`
* Augmentez les variables `threads_per_block` et/ou `blocks_per_grid`
* Supprimez ou commentez l'appel `cuda.synchronize()`

### Résultats

Dans l'exemple ci-dessus, étant donné que le noyau est écrit de telle sorte que chaque thread fonctionne sur exactement un élément de données, il est essentiel que le nombre de threads dans la grille soit égal au nombre d'éléments de données.

En **réduisant le nombre de threads dans la grille**, soit en réduisant le nombre de blocs, et/ou en réduisant le nombre de threads par bloc, il y a des éléments où le travail n'est pas effectué et nous pouvons donc voir dans la sortie que les éléments vers la fin du tableau `d_out` n'ont pas eu de valeurs ajoutées. Si vous avez modifié la configuration d'exécution en réduisant le nombre de threads par bloc, alors en fait il y a d'autres éléments dans le tableau `d_out` qui n'ont pas été traités.

**Augmenter la taille de la grille** crée en fait des problèmes d'accès à la mémoire hors limites. Cette erreur n'apparaîtra pas dans votre code pour le moment, mais plus loin dans cette section, vous apprendrez comment exposer cette erreur à l'aide de `cuda-memcheck` et la déboguer.

Vous auriez pu vous attendre à ce que la **suppression du point de synchronisation** ait entraîné une impression indiquant qu'aucun travail ou moins de travail n'avait été effectué. C'est une hypothèse raisonnable puisque sans point de synchronisation, le processeur fonctionnera de manière asynchrone pendant que le GPU est en cours de traitement. Le détail à apprendre ici est que les copies de mémoire portent une synchronisation implicite, rendant l'appel à « cuda.synchronize » ci-dessus inutile.

### Exercice : accélérer une fonction CPU en tant que noyau CUDA personnalisé

Vous trouverez ci-dessous la fonction scalaire CPU `square_device` qui pourrait être utilisée comme ufunc CPU. Votre tâche consiste à la refactoriser pour qu'elle s'exécute en tant que noyau CUDA décoré avec le décorateur `@cuda.jit`.

Vous pourriez penser que l'exécution de cette fonction sur l'appareil pourrait être beaucoup plus facile avec `@vectorize`, et vous auriez raison. Mais ce scénario vous donnera l'occasion de travailler avec toute la syntaxe que nous avons introduite avant de passer à des exemples plus complexes et plus réalistes.

Dans cet exercice, vous devrez :
* Refactoriser la définition `square_device` pour qu'elle soit un noyau CUDA qui effectuera le travail d'un thread sur un seul élément.
* Refactoriser les tableaux `d_a` et `d_out` ci-dessous pour qu'ils soient des tableaux de périphériques CUDA.
* Modifier les variables `blocks` et `threads` pour qu'elles s'adaptent aux valeurs fournies pour le `n`.
* Refactorisez l'appel à `square_device` pour qu'il soit un lancement de noyau incluant une configuration d'exécution.

Le test d'assertion ci-dessous échouera jusqu'à ce que vous implémentiez avec succès ce qui précède.

In [ ]:
# Refactor to be a CUDA kernel doing one thread's work.
# Don't forget that when using `@cuda.jit`, you must provide an output array as no value will be returned.
def square_device(a):
    return a**2

In [ ]:
# Leave the values in this cell fixed for this exercise
n = 4096

a = np.arange(n)
out = a**2 # `out` will only be used for testing below

In [ ]:
d_a = a                  # TODO make `d_a` a device array
d_out = np.zeros_like(a) # TODO: make d_out a device array

# TODO: Update the execution configuration for the amount of work needed
blocks = 0
threads = 0

# TODO: Launch as a kernel with an appropriate execution configuration
d_out = square_device(d_a)

In [ ]:
from numpy import testing
testing.assert_almost_equal(d_out, out)

## Une pause pour parler sur les choix de configuration pour la latence et l'exécution

Les GPU NVIDIA compatibles CUDA se composent de plusieurs [**Streaming Multiprocessors**](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#hardware-implementation), ou **SM** sur une matrice, avec une mémoire DRAM attachée. Les SM contiennent toutes les ressources nécessaires à l'exécution du code du noyau, y compris de nombreux cœurs CUDA. Lorsqu'un noyau est lancé, chaque bloc est attribué à un seul SM, avec potentiellement de nombreux blocs attribués à un seul SM. Les SM partitionnent les blocs en sous-divisions supplémentaires de 32 threads appelées **warps** et ce sont ces warps qui reçoivent des instructions parallèles à exécuter.

Lorsqu'une instruction prend plus d'un cycle d'horloge pour se terminer (ou dans le jargon CUDA, pour **expirer**), le SM peut continuer à effectuer un travail significatif *s'il dispose de warps supplémentaires prêts à recevoir de nouvelles instructions.* En raison des fichiers de registre très volumineux sur les SM, il n'y a pas de pénalité de temps pour qu'un SM change de contexte entre l'envoi d'instructions à un warp ou à un autre. En bref, la latence des opérations peut être masquée par les SM avec d'autres tâches significatives tant qu'il y a d'autres tâches à effectuer.

**Par conséquent, afin d'exploiter tout le potentiel du GPU et écrire des applications accélérées performantes, il est essentiel de donner aux SM la possibilité de masquer la latence en leur fournissant un nombre suffisant de warps, ce qui peut être accompli le plus simplement possible en exécutant des noyaux avec des dimensions de grille et de bloc suffisamment grandes.**

Déterminer la meilleure taille pour la grille de threads CUDA est un problème complexe, et dépend à la fois de l'algorithme et de la [capacité de calcul](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capabilities) spécifique du GPU, mais voici quelques heuristiques très approximatives que nous avons tendance à suivre et qui peuvent bien fonctionner pour commencer :

* La taille d'un bloc doit être un multiple de 32 threads (la taille d'une chaîne), avec des tailles de bloc typiques comprises entre 128 et 512 threads par bloc.
* La taille de la grille doit garantir que le GPU complet est utilisé lorsque cela est possible. Lancer une grille où le nombre de blocs est 2 à 4 fois supérieur au nombre de SM sur le GPU est un bon point de départ. Une plage de 20 à 100 blocs est généralement un bon point de départ.
* La surcharge de lancement du noyau CUDA augmente avec le nombre de blocs, donc lorsque la taille d'entrée est très importante, nous trouvons préférable de ne pas lancer une grille où le nombre de threads est égal au nombre d'éléments d'entrée, ce qui entraînerait un nombre énorme de blocs. Au lieu de cela, nous utilisons un modèle sur lequel nous allons maintenant porter notre attention pour gérer les entrées volumineuses.

## Travailler sur les plus grands ensembles de données avec les boucles Grid Stride

Les diapositives suivantes donnent un aperçu général d'une technique appelée **boucle Grid Stride** qui permet de créer des noyaux flexibles où chaque thread est capable de travailler sur plusieurs éléments de données, une technique essentielle pour les grands ensembles de données. Exécutez la cellule pour charger les diapositives.

In [ ]:
from IPython.display import IFrame
IFrame('https://view.officeapps.live.com/op/view.aspx?src=https://developer.download.nvidia.com/training/courses/C-AC-02-V1/AC_CUDA_Python_2.pptx', 640, 390)

## Une première boucle Grid Stride

Refactorisons le `add_kernel` ci-dessus pour utiliser une boucle Grid Stride afin de pouvoir le lancer pour travailler sur des ensembles de données plus volumineux de manière flexible tout en bénéficiant des avantages de la **fusion de mémoire** globale, qui permet aux threads parallèles d'accéder à la mémoire par blocs contigus, un scénario que le GPU peut exploiter pour réduire le nombre total d'opérations de mémoire :

In [ ]:
from numba import cuda

@cuda.jit
def add_kernel(x, y, out):


    start = cuda.grid(1)

    # This calculation gives the total number of threads in the entire grid
    stride = cuda.gridsize(1)   # 1 = one dimensional thread grid, returns a single value.
                                # This Numba-provided convenience function is equivalent to
                                # `cuda.blockDim.x * cuda.gridDim.x`

    # This thread will start work at the data element index equal to that of its own
    # unique index in the grid, and then, will stride the number of threads in the grid each
    # iteration so long as it has not stepped out of the data's bounds. In this way, each
    # thread may work on more than one data element, and together, all threads will work on
    # every data element.
    for i in range(start, x.shape[0], stride):
        # Assuming x and y inputs are same length
        out[i] = x[i] + y[i]

In [ ]:
import numpy as np

n = 100000 # This is far more elements than threads in our grid
x = np.arange(n).astype(np.int32)
y = np.ones_like(x)

d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_out = cuda.device_array_like(d_x)

threads_per_block = 128
blocks_per_grid = 30

In [ ]:
add_kernel[blocks_per_grid, threads_per_block](d_x, d_y, d_out)
print(d_out.copy_to_host()) # Remember, memory copy carries implicit synchronization

### Exercice : implémenter une boucle Grid Stride

Refactorisez la fonction scalaire CPU `hypot_stride` suivante pour qu'elle s'exécute en tant que noyau CUDA en utilisant une boucle Grid Stride.

In [ ]:
from math import hypot

def hypot_stride(a, b, c):
    c = hypot(a, b)

In [ ]:
# You do not need to modify the contents in this cell
n = 1000000
a = np.random.uniform(-12, 12, n).astype(np.float32)
b = np.random.uniform(-12, 12, n).astype(np.float32)
d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.device_array_like(d_b)

blocks = 128
threads_per_block = 64

hypot_stride[blocks, threads_per_block](d_a, d_b, d_c)

In [ ]:
from numpy import testing
# This assertion will fail until you successfully implement the hypot_stride kernel above
testing.assert_almost_equal(np.hypot(a,b), d_c.copy_to_host(), decimal=5)

## Chronométrage du noyau

Prenons le temps de faire quelques chronométrages de performances pour le noyau `hypot_stride`.

### Base de référence du processeur

Commençons par obtenir une base de référence avec `np.hypot` :

In [ ]:
%timeit np.hypot(a, b)

### Numba sur le CPU

Voyons maintenant une version optimisée pour le CPU :

In [ ]:
from numba import jit

@jit
def numba_hypot(a, b):
    return np.hypot(a, b)

In [ ]:
%timeit numba_hypot(a, b)

### Thread unique sur l'appareil

Juste pour voir, lançons notre noyau dans une grille avec un seul thread. Ici, nous utiliserons `%time`, qui n'exécute l'instruction qu'une seule fois pour garantir que notre mesure n'est pas affectée par la profondeur finie de la file d'attente du noyau CUDA. Nous ajouterons également un `cuda.synchronize` pour être sûrs de ne pas obtenir de temps inexacts en raison du retour du contrôle au processeur, où se trouve le minuteur, avant la fin du noyau :

In [ ]:
%time hypot_stride[1, 1](d_a, d_b, d_c); cuda.synchronize()

J'espère que ce n'est pas trop surprenant que cela soit bien plus lent que l'exécution de base du processeur.

### Parallelisation dans le GPU

In [ ]:
%time hypot_stride[128, 64](d_a, d_b, d_c); cuda.synchronize()

C'est bien plus rapide !

## Opérations atomiques et évitement des conditions de concurrence

CUDA, comme de nombreux frameworks d'exécution parallèle à usage général, permet d'avoir des conditions de concurrence dans votre code. Une condition de concurrence dans CUDA survient lorsque des threads lisent ou écrivent à partir d'un emplacement mémoire qui peut être modifié par un autre thread indépendant. En règle générale, vous devez vous soucier des :

* risques de lecture après écriture : un thread lit un emplacement mémoire en même temps qu'un autre thread peut y écrire.
* risques d'écriture après écriture : deux threads écrivent dans le même emplacement mémoire et une seule écriture sera visible lorsque le noyau sera terminé.

Une stratégie courante pour éviter ces deux risques consiste à organiser votre algorithme de noyau CUDA de telle sorte que chaque thread ait la responsabilité exclusive de sous-ensembles uniques d'éléments de tableau de sortie et/ou de ne jamais utiliser le même tableau pour l'entrée et la sortie dans un seul appel de noyau. (Les algorithmes itératifs peuvent utiliser une stratégie de double mise en mémoire tampon si nécessaire et changer les tableaux d'entrée et de sortie à chaque itération.)

Cependant, il existe de nombreux cas où différents threads doivent combiner les résultats. Considérez quelque chose de très simple, comme : « chaque thread incrémente un compteur global ». L'implémentation de ceci dans votre noyau nécessite que chaque thread :

1. Lire la valeur actuelle d'un compteur global.

2. Calculer `counter + 1`.

3. Réécrire cette valeur dans la mémoire globale.

Cependant, il n'y a aucune garantie qu'un autre thread n'ait pas modifié le compteur global entre les étapes 1 et 3. Pour résoudre ce problème, CUDA fournit des **opérations atomiques** qui liront, modifieront et mettront à jour un emplacement mémoire en une seule étape indivisible. Numba prend en charge plusieurs de ces fonctions, [décrites ici](http://numba.pydata.org/numba-doc/dev/cuda/intrinsics.html#supported-atomic-operations).

Créons notre noyau de compteur de threads :

In [ ]:
@cuda.jit
def thread_counter_race_condition(global_counter):
    global_counter[0] += 1  # This is bad

@cuda.jit
def thread_counter_safe(global_counter):
    cuda.atomic.add(global_counter, 0, 1)  # Safely add 1 to offset 0 in global_counter array

In [ ]:
# This gets the wrong answer
global_counter = cuda.to_device(np.array([0], dtype=np.int32))
thread_counter_race_condition[64, 64](global_counter)

print('Should be %d:' % (64*64), global_counter.copy_to_host())

In [ ]:
# This works correctly
global_counter = cuda.to_device(np.array([0], dtype=np.int32))
thread_counter_safe[64, 64](global_counter)

print('Should be %d:' % (64*64), global_counter.copy_to_host())

## Exercice

L'exercice suivant vous demandera d'utiliser tout ce que vous avez appris jusqu'à présent. Aucun code de solution ne sera disponible.

### Écrire un noyau pour une fonction d'histogramme

Pour cette évaluation, vous allez créer un noyau pour un histogramme. Celui-ci prendra un tableau de données d'entrée, une plage et un certain nombre de bacs, et comptera le nombre d'éléments de données d'entrée qui se trouvent dans chaque bac. Vous trouverez ci-dessous une implémentation CPU fonctionnelle de l'histogramme pour servir d'exemple pour votre travail :

In [ ]:
def cpu_histogram(x, xmin, xmax, histogram_out):
    '''Increment bin counts in histogram_out, given histogram range [xmin, xmax).'''
    # Note that we don't have to pass in nbins explicitly, because the size of histogram_out determines it
    nbins = histogram_out.shape[0]
    bin_width = (xmax - xmin) / nbins

    # This is a very slow way to do this with NumPy, but looks similar to what you will do on the GPU
    for element in x:
        bin_number = np.int32((element - xmin)/bin_width)
        if bin_number >= 0 and bin_number < histogram_out.shape[0]:
            # only increment if in range
            histogram_out[bin_number] += 1

In [ ]:
x = np.random.normal(size=10000, loc=0, scale=1).astype(np.float32)
xmin = np.float32(-4.0)
xmax = np.float32(4.0)
histogram_out = np.zeros(shape=10, dtype=np.int32)

cpu_histogram(x, xmin, xmax, histogram_out)

histogram_out

À l’aide d’une boucle Grid Stride et d’opérations atomiques, implémentez votre solution dans la cellule ci-dessous.

In [ ]:
@cuda.jit
def cuda_histogram(x, xmin, xmax, histogram_out):
    '''Increment bin counts in histogram_out, given histogram range [xmin, xmax).'''

    pass  # Replace this with your implementation

In [ ]:
d_x = cuda.to_device(x)
d_histogram_out = cuda.to_device(np.zeros(shape=10, dtype=np.int32))

blocks = 128
threads_per_block = 64

cuda_histogram[blocks, threads_per_block](d_x, xmin, xmax, d_histogram_out)

In [ ]:
# This assertion will fail until you correctly implement `cuda_histogram`
np.testing.assert_array_almost_equal(d_histogram_out.copy_to_host(), histogram_out, decimal=2)

# Gardez votre réponse, elle pourra être soumise pour la certification NVIDIA.